Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=51e70b20fd373959170a3a26e56fdcbbea562cc26e1017b0168f7253a22aac88
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-09-11 20:27:05,758 INFO numerapi.utils: starting download
./numerai_dataset_281.zip: 100%|█████████▉| 420M/422M [00:22<00:00, 21.0MB/s]2021-09-11 20:27:27,963 INFO numerapi.base_api: unzipping file...
./numerai_dataset_281.zip: 422MB [00:39, 21.0MB/s]                           

In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...


./numerai_dataset_281.zip: 422MB [01:18, 5.34MB/s]


# All Loaded...
CPU times: user 50.1 s, sys: 8.29 s, total: 58.4 s
Wall time: 1min 1s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
%%time

model = LGBMRegressor(random_state=rand)
model.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

y_pred_model = model.predict(tournament_data[feature_names])
gc.collect()

tournament_data[PREDICTION_NAME] = y_pred_model


CPU times: user 2min 10s, sys: 1.86 s, total: 2min 12s
Wall time: 24 s


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488480
n000920ed083903f    0.487922
n0038e640522c4a6    0.516811
n004ac94a87dc54b    0.499933
n0052fe97ea0c05f    0.502267
                      ...   
nffc4b682406f025    0.536144
nffd01206529f038    0.499172
nffde5030c22c56e    0.486173
nffe08336d93f198    0.497109
nffe854bb7437815    0.491831
Name: prediction, Length: 1811632, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [ ]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488480
n000920ed083903f    0.487922
n0038e640522c4a6    0.516811
n004ac94a87dc54b    0.499933
n0052fe97ea0c05f    0.502267
                      ...   
nffc4b682406f025    0.536144
nffd01206529f038    0.499172
nffde5030c22c56e    0.486173
nffe08336d93f198    0.497109
nffe854bb7437815    0.491831
Name: prediction, Length: 1811632, dtype: float64

In [ ]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


In [ ]:
%%time
tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

2021-09-11 20:30:32,793 INFO numexpr.utils: NumExpr defaulting to 8 threads.


CPU times: user 3min 58s, sys: 52.3 s, total: 4min 50s
Wall time: 52.6 s


In [ ]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.549430
n000920ed083903f    0.497103
n0038e640522c4a6    0.532488
n004ac94a87dc54b    0.571976
n0052fe97ea0c05f    0.565678
                      ...   
nffc4b682406f025    0.720801
nffd01206529f038    0.539894
nffde5030c22c56e    0.506638
nffe08336d93f198    0.519486
nffe854bb7437815    0.539451
Name: prediction, Length: 1811632, dtype: float64

In [ ]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.05)

In [ ]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

2021-09-05 16:40:56,657 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-09-05 16:40:56,658 INFO numexpr.utils: NumExpr defaulting to 8 threads.


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.453694
n000920ed083903f    0.416372
n0038e640522c4a6    0.500831
n004ac94a87dc54b    0.542277
n0052fe97ea0c05f    0.558074
                      ...   
nffcc555b6bc06aa    0.562053
nffcfd14874a15c4    0.402513
nffdb0b5a746c4f7    0.500285
nffecef421ac6791    0.501633
nfff39308bdddb48    0.523916
Name: prediction, Length: 1806285, dtype: float64

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('BeeChain.csv')

!cp BeeChain.csv "drive/My Drive/BeeChain Foundation/"